In [1]:
# EXPORT
# --- Must haves ---
import os, sys
sys.path.append('..')

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.cuda as cuda
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from surrogates4sims.mantaflowDatasets import MantaFlowDataset, getSingleSim, createMantaFlowTrainTest

from surrogates4sims.utils import create_opt, create_one_cycle, find_lr, printNumModelParams, \
                                    rmse, writeMessage, plotSampleWprediction, plotSampleWpredictionByChannel, \
                                    plotSample, curl, jacobian, stream2uv, create_movie, convertSimToImage

from surrogates4sims.models import Generator, Encoder, AE_no_P, AE_xhat_z, AE_xhat_zV2

from surrogates4sims.train import trainEpoch, validEpoch

import numpy as np
from tqdm import tqdm
from copy import deepcopy

In [2]:
# data 
eval_only=True
DEBUG = False
# model name, for tensorboard recording and checkpointing purposes.
versionName = "plateau_train"

# GPU Numbers to use. Comma seprate them for multi-GPUs.
gpu_ids = "1,2"
versionName = versionName + '_GPUs{}'.format(gpu_ids.replace(',',''))
# path to load model weights.
pretrained_path = None

# rate at which to record metrics. (number of batches to average over when recording metrics, e.g. "every 5 batches")
tensorboard_rate = 5

# number of epochs to train. This is defined here so we can use the OneCycle LR Scheduler.
epochs = 1000

# Data Directory
dataDirec = '/data/mantaFlowSim/data/smoke_pos21_size5_f200/v'
reverseXY = False 

# checkpoint directory
cps = 'cps'
tensorboard_direc = "tb"

findLRs = False  

# hyper-params
seed = 1234
np.random.seed(seed)
testSplit = .1
bz = 20
numSamplesToKeep = 400 #if not debugging
latentDim = 16
window_size = 5
filters = 128
num_conv = 4 # breaks when less than 2
simLen = 200
stack = True
simVizIndex = 0 # sim in the test set to visualize
createStreamFcn = False
doJacobian = False
repeat = 0
skip_connection = False
patience = 2
if DEBUG:
    epochs = 10000
    numSamplesToKeep = bz
    
versionName = versionName + '_latentDim{}_filters{}_bz{}_numConv{}_stream{}_jacobian{}_epochs{}_stack{}'.format(latentDim,filters,bz,num_conv,createStreamFcn,doJacobian,epochs,stack)
versionName


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=gpu_ids
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

trainData, testData = createMantaFlowTrainTest(dataDirec,simLen,testSplit,seed)
print((len(trainData),len(testData)))

Using device: cuda
(19000, 2000)


In [3]:
class MantaFlowDataset(Dataset):
    def __init__(self, 
                 dataDirec='/home/widemann1/carbon_capture/surrogate_nn_for_pde/deep-fluids/data/smoke_pos21_size5_f200/v',
                 numToKeep=np.infty,transform=None, reverseXY=False, preprocess=True, AE=False,
                 w = 1, simLen = 200): 
        if type(dataDirec) == list:
            self.files = dataDirec
        else:
            self.files = glob(os.path.join(dataDirec,'*.npz'))
        self.dataDirec = dataDirec
        self.numToKeep = numToKeep
        self.transform = transform
        self.reverseXY = reverseXY
        self.AE = AE
        self.w = w
        self.simLen = simLen
        self.data = []
 
        if numToKeep < len(self.files):
            self.files = self.files[:numToKeep]
        for f in tqdm(self.files):
            X,y = self.loadfile(f)
            
            if preprocess:
                X,y = self.preprocessFcn(X,y)
                
            if reverseXY:
                self.data.append((y,X))
            else:
                self.data.append((X,y))

    def loadfile(self,fn):
        A = np.load(fn)
        X = A['x'].astype('float32')
        X = np.rollaxis(X,-1)
        y = A['y'].astype('float32')
        return X,y

    def preprocessFcn(self,X,y):
        x_range = 11.953
        X /= x_range
        y_range = [[0.2, 0.8], [0.04, 0.12], [0.0, 199.0]]
        for i, ri in enumerate(y_range):
            y[i] = (y[i]-ri[0]) / (ri[1]-ri[0]) * 2 - 1
        return X,y
    
    def __len__(self):
        return len(self.data)

    def plot(self,idx,savefig=False):
        X, label  = self.data[idx]
        if self.reverseXY:
            X = label
            
        plt.figure(figsize=(20,10))
        
        plt.subplot(211)
        fn = self.files[idx].replace('.npz','')
        title = '{} channel 0'.format(fn)
        plt.title(title)
        plt.imshow(X[0][::-1])
        plt.colorbar()
        
        plt.subplot(212)
        title = '{} channel 1'.format(fn)
        plt.title(title)
        plt.imshow(X[1][::-1])
        plt.colorbar()
        
        if savefig:
            title = title.replace(' ','_') + '.png'
            plt.savefig(title, dpi=300)
            plt.close()
        else:
            plt.show()
    
    def __getitem__(self, idx):
        q = idx // self.simLen
        r_idx = np.random.randint(0,self.simLen-self.w)
        x = self.data[q*simLen + r_idx : q*simLen + r_idx + 1]
        y = self.data[q*simLen + r_idx + 1 : q*simLen + r_idx + 1 + self.w]
        # to unpack this data into X (image) and p (cfg settings) arrays, use the following code
        U_x, p_x = zip(*x)
        U_y, p_y = zip(*y)
        return np.array(U_x), np.array(U_y), np.array(p_x), np.array(p_y)

In [4]:
# datasets may be smaller because: numSamplesToKeep 
testDataset = MantaFlowDataset(testData, reverseXY=reverseXY, numToKeep=numSamplesToKeep, AE=False,
                               w=window_size, simLen=200)
trainDataset = MantaFlowDataset(trainData, reverseXY=reverseXY,numToKeep=numSamplesToKeep, AE=False,
                                w=window_size, simLen=200)
len(trainDataset), len(testDataset)

trainDataLoader = DataLoader(dataset=trainDataset, batch_size=bz, shuffle=True, drop_last=True)
testDataLoader = DataLoader(dataset=testDataset, batch_size=bz)

U_x, U_y, p_x, p_y = next(iter(trainDataLoader))
print(U_x.shape, p_x.shape)
U_x = U_x.squeeze()
p_x = p_x.squeeze()
U_x.shape, U_y.shape, p_x.shape, p_y.shape

100%|██████████| 400/400 [00:00<00:00, 574.33it/s]


torch.Size([20, 1, 2, 128, 96]) torch.Size([20, 1, 3])


(torch.Size([20, 2, 128, 96]),
 torch.Size([20, 5, 2, 128, 96]),
 torch.Size([20, 3]),
 torch.Size([20, 5, 3]))

In [5]:
# Encoder 

AE_model = AE_xhat_zV2(U_x, filters, latentDim, num_conv, repeat, 
                 skip_connection, stack, conv_k=3, last_k=3, 
                 act=nn.LeakyReLU(), return_z=True, stream=createStreamFcn, device='cpu')

'''
if len(gpu_ids.split(',')) > 1:
    AE_model = nn.DataParallel(AE_model)
    
'''
Xhat,z = AE_model(U_x)
Xhat.shape, z.shape

/home/bartoldson1/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[128, 8, 6]


(torch.Size([20, 2, 128, 96]), torch.Size([20, 16]))

In [6]:
!ls '/home/widemann1/surrogates4sims/cps/'

allinOneModel_batchnorm_GPUs0_latentDim16_filters128_bz16_numConv4_streamTrue_jacobianTrue_epochs2_stackTrue_lr0.0001
allinOneModel_latentDim16_filters128_bz16_numConv4_streamTrue_jacobianTrue_epochs100_stackFalse
allinOneModel_latentDim16_filters128_bz32_numConv4_streamFalse_jacobianTrue_epochs100_stackFalse
allinOneModel_latentDim16_filters128_bz32_numConv4_streamFalse_jacobianTrue_epochs100_stackTrue
allinOneModel_latentDim16_filters128_bz32_numConv4_streamTrue_jacobianTrue_epochs100_stackFalse
allinOneModel_latentDim16_filters128_bz64_numConv2_streamTrue_jacobianTrue_epochs100_stackFalse
allinOneModel_latentDim16_filters128_bz64_numConv4_streamTrue_jacobianTrue_epochs2_stackFalse
allinOneModel_latentDim16_filters128_bz8_numConv4_streamFalse_jacobianTrue_epochs100_stackFalse
allinOneModel_latentDim16_filters128_bz8_numConv4_streamFalse_jacobianTrue_epochs100_stackTrue
allinOneModel_latentDim16_filters32_bz128_numConv2_streamFalse_jacobianFalse_epochs100_stackTrue
allinOneModel_laten

In [7]:
versionName

'plateau_train_GPUs12_latentDim16_filters128_bz20_numConv4_streamFalse_jacobianFalse_epochs1000_stackTrue'

In [8]:
AE_model.load_state_dict(torch.load(os.path.join('/home/widemann1/surrogates4sims/cps',
'plateau_train_GPUs2_latentDim16_filters128_bz16_numConv4_streamFalse_jacobianFalse_epochs1000_stackTrue_lr0.0001')))

<All keys matched successfully>

In [9]:
# LIN Model
class MLP(nn.Module):
    def __init__(self, X, hiddenLayerSizes = [1024], activation=nn.ELU()):
        super(MLP,self).__init__()
        
        self.activation = activation
        self.inputSize = X.shape[1:]
        self.modules = []
        self.modules.append(nn.Linear(np.prod(self.inputSize),hiddenLayerSizes[0]))
        self.modules.append(self.activation)
        for idx,sz in enumerate(hiddenLayerSizes[:-1]):
            self.modules.append(nn.Linear(hiddenLayerSizes[idx],hiddenLayerSizes[idx+1]))
            self.modules.append(self.activation)
                               
        self.modules.append(nn.Linear(hiddenLayerSizes[-1],np.prod(self.inputSize)))
        self.layers = nn.Sequential(*self.modules)
                                
        
    def forward(self,x):
        x = self.layers(x)
        return x
    
hiddenLayers = [128,128]
LIN_model = MLP(z, hiddenLayerSizes=hiddenLayers, activation=nn.ELU())
'''
if len(gpu_ids.split(',')) > 1:
    LIN_model = nn.DataParallel(LIN_model)
'''

"\nif len(gpu_ids.split(',')) > 1:\n    LIN_model = nn.DataParallel(LIN_model)\n"

In [10]:
# LIN_model(torch.ones((1,16),device=device)).shape

In [11]:
# surrogate class

class Surrogate(nn.Module):
    
    def __init__(self, window,
                 z_size, p_size,
                LIN, encoder, decoder):
        super(Surrogate, self).__init__()
        self.window = window
        self.z_size = z_size # this does not include the size of p
        self.p_size = p_size
        self.c_size = z_size + p_size # this does include the size of p
        self.LIN = LIN
        self.encoder = encoder
        self.decoder = decoder
        
    def encode(self, U):
        
        return self.encoder(U)
        
    def decode(self, encoding):
        
        return self.decoder(encoding)
        
    def predict_next_w_encodings(self, encoding, p_y, window = None):
        '''
        use the LIN to predict the next w encodings for each 
        encoded U in the batch
        '''
        
        if window == None:
            window = self.window
            
        predicted_encodings = []
            
        # given a batch of encodings, advance each encoding window time steps.
        # save the result at each time step
        for i in range(window):
            encoding = self.LIN(encoding) + encoding # use LIN to predict delta in encoding
            # this was encoding[:,:,-self.p_size:] in 09_manta..., why the extra dimension?
            encoding[:,-self.p_size:] = p_y[:, i]
            predicted_encodings.append(encoding)
            
            
        return torch.stack(predicted_encodings)
    
    def forward(self, U, p_y):
        
        encoding = self.encode(U)
        encoding_w = self.predict_next_w_encodings(encoding, p_y)
        # want to have this agree with U_y, which is [batch_size, window_size, channels, nx, ny]
        # right now, it's [window_size, batch_size, c_size], so transpose dimensions 0 and 1
        # print(encoding_w.shape)
        U = torch.stack([self.decode(encoding_i) for encoding_i in encoding_w])
        
        return U.transpose(0,1)
    
    
surrogate = Surrogate(window_size, latentDim - 3, 3, LIN_model, AE_model.encoder, AE_model.generator)

In [12]:
# note the important difference here
foo = torch.tensor([[1,1],[2,2],[3,3]])
foo, foo.reshape(2,3), foo.transpose(0,1)

(tensor([[1, 1],
         [2, 2],
         [3, 3]]),
 tensor([[1, 1, 2],
         [2, 3, 3]]),
 tensor([[1, 2, 3],
         [1, 2, 3]]))

In [13]:
encoding = surrogate.encode(U_x)
decoding = surrogate.decode(encoding)
assert surrogate.c_size == 16
assert surrogate.p_size == len(p_x[0])
assert encoding.shape[-1] == surrogate.c_size
assert decoding.shape == U_x.shape

In [14]:
U_hat = surrogate.forward(U_x, p_y)
U_hat.shape, U_y.shape, torch.norm(U_hat-U_y,p=1)/torch.norm(U_y)

(torch.Size([20, 5, 2, 128, 96]),
 torch.Size([20, 5, 2, 128, 96]),
 tensor(367.6299, grad_fn=<DivBackward0>))

In [15]:
del surrogate, encoding, decoding, U_hat
surrogate = Surrogate(window_size, latentDim - 3, 3, LIN_model, AE_model.encoder, AE_model.generator).to(device)

In [16]:
surrogate.encoder.conv1.weight[-1]

tensor([[[-0.0615,  0.2370,  0.0193],
         [-0.1209, -0.1067,  0.1109],
         [-0.0105,  0.0456, -0.2153]],

        [[-0.0179,  0.2263, -0.1354],
         [ 0.0473, -0.0644, -0.1862],
         [-0.0807,  0.0856,  0.0642]]], device='cuda:0',
       grad_fn=<SelectBackward>)

In [17]:
max_lr = .0001
start_lr = 5*max_lr/10
#opt = create_opt(max_lr,model)
#lr_scheduler = create_one_cycle(opt,max_lr,epochs,trainDataLoader)
opt = torch.optim.Adam(surrogate.parameters(),lr=max_lr,betas=(.5,.999))
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,patience=patience)

In [18]:
def L1_loss(pred, target):
    return torch.mean(torch.abs(pred - target))


def jacobian_loss(pred, target, device='cpu'):
    return L1_loss(jacobian(pred, device), jacobian(target, device))


def curl_loss(pred, target, device):
    return L1_loss(curl(pred, device), curl(target, device))


L = nn.MSELoss()


def p_loss(pred, target):
    return L(pred[:, -target.shape[1]:], target)


def loss(pred, target, device):
    
    if createStreamFcn:
        pred = stream2uv(pred, device)
        
    L1 = L1_loss(pred, target)
    Lj = 0
    if doJacobian:
        Lj = jacobian_loss(pred, target, device)
        
    return L1 + Lj

In [19]:
def trainEpoch(myDataLoader, tensorboard_writer, model, opt, p_loss, loss,
               metric, lr_scheduler, tensorboard_rate, device,
               tensorboard_recorder_step, total_steps):
    running_loss = 0.0
    running_rmse = 0.0
    total_loss = 0.0
    running_ploss = 0.0
    for i, sampleBatch in enumerate(myDataLoader, start=1):

        # --- Main Training ---
        
        # gpu
        U_x, U_y, p_x, p_y = sampleBatch
        U_x = U_x.squeeze().to(device)
        p_x = p_x.squeeze().to(device)
        U_y = U_y.to(device)
        p_y = p_y.to(device)
            

        # zero the parameter gradients
        opt.zero_grad()

        U_hat = model(U_x, p_y)
        pl = 0
        ll = loss(U_hat, U_y, device)
        combined_loss = pl + ll
        combined_loss.backward()
        opt.step()
        
        # loss
        batch_loss = combined_loss.item()
        running_loss += batch_loss
        total_loss += batch_loss
        
        batch_ploss = pl
        running_ploss += batch_ploss

        # --- Metrics Recording ---

        # metrics
        r = metric(U_hat, U_y)
        running_rmse += r

        # record lr change
        total_steps += 1
        tensorboard_writer.add_scalar(tag="LR", scalar_value=opt.param_groups[0]['lr'], global_step=total_steps)
        #lr_scheduler.step()

        # tensorboard writes
        if (i % tensorboard_rate == 0):
            tensorboard_recorder_step += 1
            avg_running_loss = running_loss/tensorboard_rate
            avg_running_rmse = running_rmse/tensorboard_rate
            avg_running_ploss = running_ploss/tensorboard_rate
            tensorboard_writer.add_scalar(tag="Loss", scalar_value=avg_running_loss, global_step=tensorboard_recorder_step)
            tensorboard_writer.add_scalar(tag="p_loss", scalar_value=avg_running_ploss, global_step=tensorboard_recorder_step)
            tensorboard_writer.add_scalar(tag=metric.__name__, scalar_value=avg_running_rmse, global_step=tensorboard_recorder_step)
            # reset running_loss for the next set of batches. (tensorboard_rate number of batches)
            running_loss = 0.0
            running_rmse = 0.0
            running_ploss = 0.0

    return total_loss, tensorboard_recorder_step, total_steps

In [20]:
try:
    os.mkdir(cps)
except:
    print("checkpoints directory already exists :)")
    
# create a summary writer.
train_writer = SummaryWriter(os.path.join(tensorboard_direc, versionName,'train'))
test_writer = SummaryWriter(os.path.join(tensorboard_direc, versionName,'valid'))
tensorboard_recorder_step = 0
total_steps = 0

checkpoints directory already exists :)


In [21]:
writeMessage('---------- Started Training ----------', versionName)
bestLoss = np.infty

if not eval_only:
    for epoch in tqdm(range(1, epochs+1)):  # loop over the dataset multiple times

        writeMessage("--- Epoch {0}/{1} ---".format(epoch, epochs), versionName)

        surrogate.train()
        trainLoss, tensorboard_recorder_step, total_steps = trainEpoch(trainDataLoader, 
                                                                       train_writer, surrogate,
                                                                       opt, p_loss, loss,
                                                                       rmse, lr_scheduler, 
                                                                       tensorboard_rate, device,
                                                                       tensorboard_recorder_step, total_steps)

        writeMessage("trainLoss: {:.4e}".format(trainLoss),versionName)
        writeMessage("LR: {:.4e}".format(opt.param_groups[0]['lr']),versionName)
        if trainLoss < bestLoss:
            bestLoss = trainLoss
            writeMessage("Better trainLoss: {:.4e}, Saving models...".format(bestLoss),versionName)
            torch.save(surrogate.state_dict(), os.path.join(cps,versionName))

    #     model.eval()
    #     valLoss = validEpoch(testDataLoader, test_writer, model, p_loss, loss, rmse, device, tensorboard_recorder_step)
    #     writeMessage("valLoss: {:.4e}".format(valLoss),versionName)

        # checkpoint progress
    #     if valLoss < bestLoss:
    #         bestLoss = valLoss
    #         writeMessage("Better valLoss: {:.4e}, Saving models...".format(bestLoss),versionName)
    #         torch.save(model.state_dict(), os.path.join(cps,versionName))

        lr_scheduler.step(trainLoss)

        if opt.param_groups[0]['lr'] < 1e-8:
            break
    writeMessage('---------- Finished Training ----------', versionName)

  0%|          | 0/1000 [00:00<?, ?it/s]

---------- Started Training ----------
--- Epoch 1/1000 ---


/home/bartoldson1/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


trainLoss: 4.0941e-01
LR: 1.0000e-04
Better trainLoss: 4.0941e-01, Saving models...


  0%|          | 1/1000 [00:14<3:57:36, 14.27s/it]

...

trainLoss: 1.0871e-01
LR: 1.0000e-08
--- Epoch 163/1000 ---


 16%|█▋        | 163/1000 [36:53<3:09:05, 13.56s/it]

trainLoss: 1.1386e-01
LR: 1.0000e-08
--- Epoch 164/1000 ---


 16%|█▋        | 163/1000 [37:01<3:10:08, 13.63s/it]


KeyboardInterrupt: 

In [23]:
surrogate.load_state_dict(torch.load(os.path.join(cps,versionName)))

<All keys matched successfully>

In [26]:
surrogate.eval()
U_hats = []
Us = []
for i, sampleBatch in enumerate(testDataLoader, start=1):

    # gpu
    U_x, U_y, p_x, p_y = sampleBatch
    U_x = U_x.squeeze().to(device)
    p_x = p_x.squeeze().to(device)
    U_y = U_y.to(device)
    p_y = p_y.to(device)
    with torch.no_grad():
        Us.append(U_y.detach().cpu())
        
        U_hat = surrogate(U_x, p_y)
                    
        U_hats.append(U_hat.detach().cpu())
        
        
Real_U = torch.stack(Us)
#Real_X_img = convertSimToImage(Real_X)

Surr_U = torch.stack(U_hats)
#Surr_X_img = convertSimToImage(Surr_X)

/home/bartoldson1/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


In [30]:
rel_error = torch.norm(Real_U - Surr_U)/torch.norm(Real_U)
rel_error

tensor(0.5263)